In [1]:
import cv2
import numpy as np
import os

In [2]:
ds_path = "desktop\\Ceeri3_2"
rgb_path = os.path.join(ds_path , "rgb")
depth_path = os.path.join(ds_path , "depth")
img_files = os.listdir(rgb_path)
depth_files = os.listdir(depth_path)

In [3]:
for file in img_files:
    imgpath = rgb_path + "\\" + file
    img = cv2.imread(imgpath)
    img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
    img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
    hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)
    destination_path = os.path.join(ds_path, "rgb_equalise") 
    cv2.imwrite(os.path.join(destination_path,file),hist_equalization_result)
    

NameError: name 'img_to_yuv' is not defined

In [10]:
for file in depth_files:
    imgpath = depth_path + "\\" + file
    img = cv2.imread(imgpath)
    grayscale_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
#     img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
    destination_path = os.path.join(ds_path, "depth_equalise") 
    res=cv2.equalizeHist(grayscale_image)
    res = cv2.cvtColor(res, cv2.COLOR_GRAY2BGR)
    os.path.join(destination_path,file)
    cv2.imwrite(os.path.join(destination_path,file),res)
    